In [ ]:
import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
home_dir = os.getenv("HOME")
raw_data = pd.read_csv(home_dir + '/Desktop/DataPacket/data.csv')

In [ ]:
raw_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Concatenate, Input
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers.core import Activation
from keras.models import load_model

In [ ]:
data_start = 0
data_stop = 1000
data_part = raw_data[data_start:data_stop]
laser_data = data_part.drop(["steering_angle", "speed"], axis=1)
laser_data = laser_data.values
maximum_laser_range = 12.0
for i in range(len(laser_data)):
    for j in range(len(laser_data[i])):
        if np.isinf(laser_data[i][j]):
            laser_data[i][j] = maximum_laser_range
laser_data = laser_data / 12
X_laser_train, X_laser_validation = train_test_split(laser_data, test_size=0.2)
speed = data_part["speed"].values
steering_angle = data_part["steering_angle"].values
output = np.dstack((steering_angle, speed))[0]

img_input = []
h = 240
w = 320
c = 3
for i in range(data_start+1, data_stop+1):
    img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", c)
    img = cv2.resize(img, (w, h))
    img_input.append(img)
img_input = np.asarray(img_input).astype(np.float32) / 255.0
print("img input shape:",img_input.shape)

X_train, X_validation, y_train, y_validation = train_test_split(img_input, output, test_size=0.2, random_state=22)
del img_input
del steering_angle
del output
del speed
print("1st part x train shape:",X_train.shape)
print("1st part y train shape:",y_train.shape)
print("1st part laser data shape:",laser_data.shape[1])

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 2, kernel_size = (3,3),padding = 'same',activation ='relu'))
model.add(Conv2D(filters = 4, kernel_size = (3,3),padding = 'same',activation ='relu'))
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 24, kernel_size = (7,7),padding = 'same',activation ='relu'))
model.add(Flatten())

image_input = Input(shape=(h, w, c))
flatten_image_outputs = model(image_input)

laser_input = Input(shape = (laser_data.shape[1],))
concatenatedFeatures = Concatenate(axis = 1)([laser_input, flatten_image_outputs])

dense = Dense(128, activation = "relu")(concatenatedFeatures)
dense = Dense(64, activation = "relu")(dense)
dense = Dense(16, activation = "relu")(dense)
main_output = Dense(2, activation = "softmax")(dense)

drive_model = Model(inputs=[image_input,laser_input], outputs=main_output)

drive_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
hs = drive_model.fit([X_train, X_laser_train], y_train, validation_data=([X_validation, X_laser_validation], y_validation), batch_size=64, epochs=5)

In [ ]:
data_start = 1000
data_stop = 2000
for part in range(1,int(len(raw_data)/1000)+1):
    if data_stop > len(raw_data):
        data_stop = len(raw_data)
    if data_stop <= data_start:
        break
    data_part = raw_data[data_start:data_stop]
    laser_data = data_part.drop(["steering_angle", "speed"], axis=1)
    laser_data = laser_data.values
    maximum_laser_range = 12.0
    for i in range(len(laser_data)):
        for j in range(len(laser_data[i])):
            if np.isinf(laser_data[i][j]):
                laser_data[i][j] = maximum_laser_range
    laser_data = laser_data / 12
    X_laser_train, X_laser_validation = train_test_split(laser_data, test_size=0.2)
    speed = data_part["speed"].values
    steering_angle = data_part["steering_angle"].values
    output = np.dstack((steering_angle, speed))[0]
    
    img_input = []
    h = 240
    w = 320
    c = 3
    for i in range(data_start+1, data_stop+1):
        img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", c)
        img = cv2.resize(img, (w, h))
        img_input.append(img)
    img_input = np.asarray(img_input).astype(np.float32) / 255.0
    print("img input shape:",img_input.shape)
    
    X_train, X_validation, y_train, y_validation = train_test_split(img_input, output, test_size=0.2, random_state=22)
    del img_input
    del steering_angle
    del output
    del speed
    print(part+1,"th part x train shape:",X_train.shape)
    print(part+1,"th part y train shape:",y_train.shape)
    print(part+1,"th part laser data shape:",laser_data.shape[1])
    hs = drive_model.fit([X_train, X_laser_train], y_train, validation_data=([X_validation, X_laser_validation], y_validation), batch_size=64, epochs=5)
    data_start = data_start + 1000
    data_stop = data_stop + 1000

In [ ]:
import plotly
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

def trainingResultsGraph(model, modelCode):
    trace0 = go.Scatter(
        x = drive_model.epoch,
        y = drive_model.history['loss'],
        mode = 'lines',
        name = 'loss',
        line=dict(color='aquamarine')
    )

    trace1 = go.Scatter(
        x = drive_model.epoch,
        y = drive_model.history['val_loss'],
        mode = 'lines',
        name = 'val_loss',
        line=dict(color='darkred', dash='dash')
    )

    trace2 = go.Scatter(
        x = drive_model.epoch,
        y = drive_model.history['acc'],
        mode = 'lines',
        name = 'acc',
        line=dict(color='violet')
    )

    trace3 = go.Scatter(
        x = drive_model.epoch,
        y = drive_model.history['val_acc'],
        mode = 'lines',
        name = 'val_acc',
        line=dict(color='aqua', dash='dash')
    )

    updatemenus = list([
        dict(type="buttons",
             active=-1,
             buttons=list([
                dict(label = 'Acc Graph',
                     method = 'update',
                     args = [{'visible': [False, False, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation accuracy'}]),
                dict(label = 'Loss Graph',
                     method = 'update',
                     args = [{'visible': [True, True, False, False]},
                             {'title': 'Trained Model'+modelCode+' training and validation loss'}]),
                dict(label = 'Both',
                     method = 'update',
                     args = [{'visible': [True, True, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation values'}])
            ]),
        )
    ])

    data = [trace0, trace1, trace2, trace3]
    layout = dict(title='Trained Model'+modelCode+' training and validation values',
                  xaxis = dict(title = 'Epochs'),
                  updatemenus=updatemenus)

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='lossGraph')
    
trainingResultsGraph(hs, "1")

In [ ]:
try:
    import rospkg
    import rospy

    rospack = rospkg.RosPack()
    package_path = rospack.get_path('angelshark_deeplearning')
    
except ImportError:
    package_path = home_dir + "/Desktop"
    
    import os
    if not os.path.exists(package_path + "/model"):
        os.makedirs(package_path + "/model")
    
    print("Please move them to deeplearning package model directory to angelshark_deeplearning/model")
    
print("Model saved to ",str(package_path)," dir!!!")

In [ ]:
import json
import matplotlib.pylab as plt
import random
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# Save model weights and json.
drive_model.save_weights(package_path + '/model/model.h5')
model_json = drive_model.to_json()
with open(package_path + '/model/model.json', 'w') as outfile:
    json.dump(model_json, outfile)

In [ ]:
# Train and validation loss chart
print(hs.history.keys())

plt.plot(hs.history['loss'])
plt.plot(hs.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()